In [ ]:
using CSV, DataFrames, Plots, Statistics, JuMP, Gurobi

In [ ]:
pollution = DataFrame(CSV.File("Data/pollution_location.csv"))
pollution_xy = pollution[:,1:2]|>Matrix
sensors_xy = DataFrame(CSV.File("Data/sensors_location.csv"))[:,1:2]|>Matrix

lat_to_km = [111, 92]
lat_diff = (pollution_xy[:,1].-sensors_xy[:,1]').*lat_to_km[1]
long_diff = (pollution_xy[:,2].-sensors_xy[:,2]').*lat_to_km[2]

dist_matrix = sqrt.(lat_diff.^2 + long_diff.^2)'
n, m = size(dist_matrix)

(2668, 7107)

In [ ]:
q = pollution.AIQ
r = 2
p = 1  # unit price

In [ ]:
# q = f(quality)

In [ ]:
function model1(mu)
    
    model = Model(Gurobi.Optimizer);
    set_optimizer_attribute(model, "TimeLimit", 30);
    
    @variable(model, x[1:n], Bin) #Whether we put a censor at spot i
    @variable(model, y[1:m], Bin) #Whether we spot j is covered
    
    @constraint(model, y.<=x'*(dist_matrix.<=r))
    
    @objective(model, Max, mu .* sum(y.*q) - (1-mu)*p*sum(x))
    optimize!(model)
    return value.(x)
end

sum(model1(1))